In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from src.attention_analysis import AttentionAnalyzer

/home/daniil/Learn/Semestr_3/Modern_CV/Fine-tuning_1.2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Инициализация анализатора
analyzer = AttentionAnalyzer('../models/vit-fashion-mnist-fast')

OSError: Can't load image processor for '../models/vit-fashion-mnist-fast'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '../models/vit-fashion-mnist-fast' is the correct path to a directory containing a preprocessor_config.json file

In [ ]:
# Загрузка тестовых изображений
dataset = load_dataset("fashion_mnist")
test_images = dataset['test']['image'][:10]  # Первые 10 изображений
test_labels = dataset['test']['label'][:10]

label_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

In [ ]:
# Анализ для первого изображения
image_idx = 0
image = test_images[image_idx]
true_label = label_names[test_labels[image_idx]]

print(f"Анализ изображения: {true_label}")

In [ ]:
# Визуализация для разных слоев
for layer in [0, 4, 8]:
    fig = analyzer.visualize_cls_attention(image, layer_idx=layer, head_idx=0)
    fig.suptitle(f'Layer {layer} - Head 0 - {true_label}', fontsize=16)
    plt.show()

In [ ]:
# Attention Rollout для нескольких изображений
for i in range(3):
    image = test_images[i]
    true_label = label_names[test_labels[i]]
    
    fig = analyzer.attention_rollout(image)
    fig.suptitle(f'Attention Rollout - {true_label}', fontsize=16)
    plt.show()


In [ ]:
# Сравнение attention по слоям для одного изображения
analyzer.compare_heads_layers(
    test_images[0], 
    save_path='../reports/attention_visualizations/layers_comparison.png'
)

In [ ]:
# Анализ разнообразия attention heads
diversity_scores = analyzer.analyze_head_diversity(test_images[0])

print("Разнообразие attention heads по слоям:")
for layer, diversity in diversity_scores:
    print(f"Layer {layer}: {diversity:.3f}")

In [ ]:
# Визуализация
layers = [score[0] for score in diversity_scores]
diversities = [score[1] for score in diversity_scores]

plt.figure(figsize=(10, 6))
plt.plot(layers, diversities, 'o-', linewidth=2, markersize=8)
plt.xlabel('Layer')
plt.ylabel('Diversity Score')
plt.title('Diversity of Attention Heads Across Layers')
plt.grid(True, alpha=0.3)
plt.savefig('../reports/attention_visualizations/head_diversity.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Тестирование с разными размерами изображений
def test_different_resolutions():
    original_image = test_images[0]
    
    # Разные размеры
    sizes = [(56, 56), (112, 112), (224, 224)]
    
    fig, axes = plt.subplots(1, len(sizes), figsize=(15, 5))
    
    for idx, size in enumerate(sizes):
        # Ресайз изображения
        resized_image = original_image.resize(size)
        
        # Анализ внимания
        attentions, _, _ = analyzer.get_attention_maps(resized_image)
        cls_attention = attentions[-1][0].mean(dim=0)[0, 1:].cpu().numpy()
        
        grid_size = int(np.sqrt(cls_attention.shape[0]))
        attention_map = cls_attention.reshape(grid_size, grid_size)
        
        axes[idx].imshow(attention_map, cmap='hot')
        axes[idx].set_title(f'Size: {size}')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('../reports/attention_visualizations/resolution_impact.png', dpi=300, bbox_inches='tight')
    plt.show()

test_different_resolutions()

In [ ]:
# Анализ attention для разных классов
class_analysis = {}

for class_id in range(5):  # Первые 5 классов
    class_images = [img for img, lbl in zip(test_images, test_labels) if lbl == class_id]
    
    if class_images:
        # Анализ для первого изображения класса
        attention_maps = []
        for layer in [0, 6, 11]:  # Начальный, средний, конечный слои
            fig = analyzer.visualize_cls_attention(
                class_images[0], 
                layer_idx=layer, 
                head_idx=0
            )
            attention_maps.append(fig)
        
        class_analysis[label_names[class_id]] = attention_maps

In [ ]:
# Количественные метрики внимания
def calculate_attention_metrics(attention_maps):
    """Вычисление метрик внимания"""
    metrics = {}
    
    # Focus score (насколько внимание сконцентрировано)
    entropy = -torch.sum(attention_maps * torch.log(attention_maps + 1e-8), dim=-1)
    metrics['focus_score'] = 1 - (entropy / np.log(attention_maps.shape[-1]))
    
    # Coverage (сколько патчей охвачено вниманием)
    coverage = (attention_maps > attention_maps.mean()).float().mean(dim=-1)
    metrics['coverage'] = coverage
    
    return metrics

In [ ]:
# Применение к тестовым изображениям
for i in range(3):
    attentions, _, _ = analyzer.get_attention_maps(test_images[i])
    
    # Анализ последнего слоя
    last_layer_attention = attentions[-1][0].mean(dim=0)  # Усредняем по головам
    cls_attention = last_layer_attention[0, 1:]
    
    metrics = calculate_attention_metrics(cls_attention.unsqueeze(0))
    
    print(f"Image {i} ({label_names[test_labels[i]]}):")
    print(f"  Focus Score: {metrics['focus_score'].mean().item():.3f}")
    print(f"  Coverage: {metrics['coverage'].mean().item():.3f}")
    print()